# DataFrame

**OBJETIVO**: O objetivo deste notebook é mostrar os príncipais métodos de manipulação de dados utilizando o pandas e numpy

---

## Cabeçalho 

In [ ]:
import os
import requests
import zipfile

import numpy as np
import pandas as pd

from io import BytesIO
from pathlib import Path
from pyarrow.lib import ArrowInvalid

In [ ]:
%config Completer.use_jedi = False

# pode ser "completo" ou "amostra"
TIPO_DADOS = "completo"

# checa se o notebook está sendo executado no google colab
GOOGLE_COLAB = "google.colab" in str(get_ipython())

# monta a pasta com os conteúdos
if GOOGLE_COLAB:
    from google.colab import drive
    drive.mount("/content/drive")
    
# informa o caminho para a pasta de ciência de dados a partir do drive
# você pode deixar isso como vazio se você não tiver adicionado a pasta
CAMINHO_DRIVE = "Ciência de Dados"

# links para os dados a serem baixados diretamente da Web
LINKS_DADOS = {
    "amostra": {
        "escola.parquet": "https://drive.google.com/uc?id=1i51S1GKVqc-_5KgWsfXDc3Fnkf1y700y&export=download",
        "ideb.parquet": "https://drive.google.com/uc?id=1NXCGhtluNbd2Vccyof3Y-eaw1vHkvIAL&export=download",
        "turma.parquet": "https://drive.google.com/uc?id=1ks0lsbU5GXI6sbZRG9f8zGdzZqDRK8SG&export=download",
    },
    "completo": {
        "escola.parquet": "https://drive.google.com/uc?id=1_at50Wh4JJz1jR-hhsqutVaUorN0M4IL&export=download",
        "ideb.parquet": "https://drive.google.com/uc?id=183GP-MVohBC84NvHMLYcBQ4YRRCPHxYi&export=download",
        "turma.parquet": "https://drive.google.com/uc?id=1OSrAB3oAAX-9NNWZ7dG0mcWFOIcjWUs8&export=download",
    }
}

In [ ]:
vPATH_NOTEBOOK = Path(os.path.dirname(os.path.realpath("__file__")))
vPATH_DADOS = vPATH_NOTEBOOK.parent.parent / f"dados/{TIPO_DADOS}"

---

## Criação

O primeiro passo na manipulação de dados é sempre realizar o carregamento dos dados a serem trabalhados. Para isso o pandas contém um conjunto de funções que começam com read_ e que terminam com o tipo de arquivo a ser baixado.

É importante notar que o pandas é capaz de trabalhar tanto com dados locais quanto com dados Web. Veja a célula abaixo por exemplo, temos 3 opções de download dos dados: Dentro do ambiente colab, na qual referenciamos um caminho para o arquivo a partir das pastas do ambiente, um segundo para o qual baixamos diretamenta de um link da internet e um terceiro, para quando temos os arquivos no diretório local

In [ ]:
def carrega_parquet(nome: str, pasta: str, **kwargs) -> pd.DataFrame:
    global GOOGLE_COLAB, CAMINHO_DRIVE, vPATH_DADOS, TIPO_DADOS
    if GOOGLE_COLAB and CAMINHO_DRIVE != "":
        return pd.read_parquet(f"drive/MyDrive/{CAMINHO_DRIVE}/dados/{TIPO_DADOS}/{pasta}/{nome}.parquet", **kwargs)
    elif GOOGLE_COLAB or not os.path.exists(vPATH_DADOS / f"{pasta}/{nome}.parquet"):
        return pd.read_parquet(LINKS_DADOS[TIPO_DADOS][f"{nome}.parquet"])
    else:
        try:
            return pd.read_parquet(vPATH_DADOS / f"{pasta}/{nome}.parquet", **kwargs)
        except ArrowInvalid:
            return pd.read_parquet(LINKS_DADOS[TIPO_DADOS][f"{nome}.parquet"])

df1 = carrega_parquet("escola", "aquisicao", filters=[("ANO", "=", 2020)])
df2 = carrega_parquet("turma", "aquisicao", filters=[("ANO", "=", 2020)])
df3 = carrega_parquet("ideb", "aquisicao")

In [ ]:
df1

Cada função de carregamento de arquivos tem sua própria lista de parâmetros e particularidades. Frequentemente, um dos tipos principais de arquivos que iremos trabalhar são os CSV (comma separated values) que, para o contexto brasileiro, necessitam da configuração de 3 parâmetros em particular:
- sep: O tipo de separador de arquivos
- decimal: O tipo de indicador de casas decimais
- encoding: A codificação de caracteres

In [ ]:
pd.read_csv(f"drive/MyDrive/{CAMINHO_DRIVE}/aulas/03.Manipulação de Dados/gestor.CSV", sep="|", encoding="latin-1", decimal=".")

É bastante comum que seja necessário ler dados contidos em arquivos zip ou alguma outra localização que não seja um endereço. Por conta disso o pandas (e na verdade o python como um todo) também aceita Buffers além de endereços.

Buffers são como um reservatório de dados. Em um dado momento você pode abrir esse reservatório e deixar o fluxo de água (dados) fluir para um outro container (memória RAM) e assim trabalhar essa água (dados) em um local mais adequado (jupyter)

Por exemplo, para um arquivo zip o código abaixo cria um objeto ZipFile e executa a função open que criará um Buffer para o conteúdo do arquivo dentro do zip. Desta forma o pandas irá percorrer esse buffer e construir o data frame de maneira adequada

In [ ]:
with zipfile.ZipFile(f"drive/MyDrive/{CAMINHO_DRIVE}/aulas/03.Manipulação de Dados/gestor.zip") as z:
    df = pd.read_csv(z.open("gestor.CSV"), sep="|", encoding="latin-1", decimal=".")
df

Outro caso comum do uso de buffers é no download de arquivos. Por exemplo suponhamos que eu quisesse ler o mesmo zip só que agora diretamente da Web, mas sem baixar nenhum arquivo para o HD. Como estamos falando de um arquivo zip o pandas não consegue lê-lo diretamente, mas podemos baixar o conteúdo da Web dentro de um objeto BytesIO e fazer o pandas ler esse objeto

In [ ]:
resposta = requests.get("https://drive.google.com/uc?id=1ZyawTjY0fbiCwk6fsF0urF5YfNxP6Gjc&export=download")
buffer = BytesIO(initial_bytes=resposta.content)
with zipfile.ZipFile(buffer) as z:
    df = pd.read_csv(z.open("gestor.CSV"), sep="|", encoding="latin-1", decimal=".")
df

Outra maneira de inicializar um data frame é por meio da função pd.DataFrame que espera que passemos informações de nomes de linhas e colunas e os dados contidos no data frame para construí-lo

In [ ]:
pd.DataFrame(
    data=np.random.randint(low=0, high=10, size=(5, 5)),
    columns=[f"col_{i}" for i in range(5)],
    index=range(5),
)

In [ ]:
exam_data = {
    "name": ["Anastasia", "Dilma", "Katarina", "James", "Emily", "Michael", "Mateus", "Laura", "Kevin", "João"],
    "nota": [12.5, 9, 16.5, np.nan, 9, 20, 14.5, np.nan, 8, 19],
    "tentativas": [1, 3, 2, 3, 2, 3, 1, 1, 2, 1],
    "qualidade": ["sim", "não", "sim", "não", "não", "sim", "sim", "não", "não", "sim"]
}
labels = list("abcdefghij")
pd.DataFrame(exam_data, index=labels)

---

**Exercício**

Represente a gráfico abaixo como um data frame, na qual temos duas colunas: Nome do país e casos de COVID

<img src="http://cdn.statcdn.com/Infographic/images/normal/21176.jpeg" width=300 height=600/>

In [ ]:
#@title Resposta
pd.DataFrame(
    data=[
        ("Belgium", 39166),
        ("Israel", 35572),
        ("Czechia", 33103),
        ("Panama", 32300),
        ("Kuwait", 30957),
        ("United States", 28414),
        ("Peru", 28213),
        ("Chile", 27455),
        ("Argentina", 26593),
        ("Spain", 26554)
    ],
    columns=["pais", "covid"]
)

---

## Exploração de Dados 

Uma vez carregados os dados nós podemos começar a explorar o conteúdo das tabelas. No jupyter, uma primeira maneira simples é simplesmente rodar uma célula com a variável que retém o objeto

In [ ]:
df1

Outra maneira consiste em utilizar a função nativa do jupyter, display, que irá imprimir de maneira formatada o objeto a partir de qualquer ponto da execução do código

In [ ]:
a = 1
df1
b = 2

In [ ]:
a = 1
print(df1)
b = 2

In [ ]:
a = 1
display(df1)
b = 2

Algumas vezes nosso objetivo vai ser apenas explorar uma amostra dos dados disponíveis. Nós podemos fazer isso a partir dos métodos de head, tail e sample:
- head: Mostra as primeiras linhas (padrão = 5) do data frame
- tail: Mostra as últimas linhas (padrão = 5) do data frame
- sample: Mostra linhas aleatórias (padrão = 1) do data frame

In [ ]:
df1.head()

In [ ]:
df1.head(10)

In [ ]:
df1.tail()

In [ ]:
df1.tail(10)

In [ ]:
df1.sample()

In [ ]:
df1.sample(10)

Podemos, em seguida, explorar algumas características do data frame como o seu tamanho, nomes de colunas e linhas por meio dos atributos shape, columns, index e values

In [ ]:
df1.shape

In [ ]:
df1.index

In [ ]:
df1.columns

In [ ]:
df2.values

Outro método de exploração se chama "info" que nos permite verificar o número de colunas, o tipo de dados de cada coluna, o número de valores preenchidos e a memória necessária para armazenar o data frame.

É importante destacar que há uma série de tipos de dados que o pandas consegue trabalhar, destacam-se

In [ ]:
df2.info()

In [ ]:
df1.info()

In [ ]:
df1.info(memory_usage="deep")

In [ ]:
df1.describe()

---

**Exercício**

Leia os dados contidos em https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/latest/owid-covid-latest.csv e descreva nos responda o seguinte:

1. Quantos países estão registrados na base?
2. Quantas e quais informações nós temos disponíveis?
3. Quanto o dataset ocupa na memória RAM?
4. Qual o número médio de casos de COVID por pais?

In [ ]:
#@title Resposta
owid = pd.read_csv("https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/latest/owid-covid-latest.csv")

print(f"Temos {owid.shape[0]} países na base")
display(owid.info(memory_usage="deep"))
display(owid.describe())

---

## Seleções 

Em pandas uma boa parte do nosso trabalho será fazer seleções específicas de uma tabela. Dessa forma, dado a estrutura 2D, temos que aprender tanto como selecionar colunas como também linhas.

A primeira sintaxe que nós trabalhamos é a de chaves, na qual o dataframe se comporta como se fosse um dicionário, e estamos selecionando a coluna do dataframe pelo nome da mesma (que pode ser uma string, lista, datetime, etc.)

In [ ]:
df1["ID_ESCOLA"]

Note que o resultado dessa seleção, como esperado, é uma série de dados que representa a coluna selecionada.

Caso tentemos selecionar uma coluna inexistente nós receberemos um KeyError

In [ ]:
df1["ID_ESC"]

De forma análoga ao que vimos em arrays e séries, para DataFrames podemos selecionar um conjunto de colunas por meio de uma lista de colunas. Diferente da operação anterior, nós receberemos um novo DataFrame com apenas as colunas selecionadas do original

In [ ]:
df1[["ID_ESCOLA", "CO_MUNICIPIO"]]

In [ ]:
df1[["ID_ESCOLA"]]

Além da seleção por chaves, podemos utilizar a mesma sintaxe do .iloc que foi utilizada em séries

In [ ]:
df1.iloc[[1, 2, 3]]

In [ ]:
df1.iloc[:5]

In [ ]:
df1.iloc[0]

In [ ]:
df1.iloc[[0]]

Entretanto, diferente de séries, por estarmos numa estrutura 2D, podemos selecionar tanto linhas quanto colunas utilizando a "," entre os diferentes slices aplicados

In [ ]:
df1.iloc[:, [2, 3]]

In [ ]:
df1.iloc[1:4, :4]

In [ ]:
df1.iloc[0, :4]

In [ ]:
df1.iloc[:, 4]

Bom, se temos uma maneira de selecionar linhas e colunas pelo número do índice, naturalmente teremos alguma maneira de fazer a mesma coisa pelo nome, o que no caso de DataFrames se traduz na sintaxe do .loc

In [ ]:
df1.loc[1:4, ["ID_ESCOLA", "CO_MUNICIPIO", "TP_DEPENDENCIA", "TP_CATEGORIA_ESCOLA_PRIVADA"]]

*NOTA: Muitas vezes os nomes de índices de um dataframe serão os valores números de 0 a n das linhas do dataframe, entretanto é importante notar que o loc procura pelo nome do índice e não o número*

Da mesma forma que séries, podemos ainda selecionar as colunas de um DataFrame por meio de slices com o nome das colunas

In [ ]:
df1.loc[:, "CO_MUNICIPIO":"DT_ANO_LETIVO_TERMINO":2]

Podemos também selecionar linhas e colunas por meio do método reindex. A grande diferença desse método para os demais é que caso uma coluna ou linha não exista nós não receberemos um erro, ao inves disso esse campo será criado com valores nulos (ou com o valor passado pelo parâmetro fill_value)

In [ ]:
df1.reindex(columns=["ID_ESCOLA", "NÃO EXISTE"])

In [ ]:
df1.reindex(columns=["ID_ESCOLA", "NÃO EXISTE"], fill_value=0)

In [ ]:
df1.reindex(index=[0, 1, 2, "NÃO EXISTE"])

---

**Exercício**

Seleciona as colunas do tipo "IN_INTERNET" do data frame de escola e reponda, para cada uma delas:

1. Quantas escolas disponibilizaram dados?
2. Quantos % das escolas tem acesso?

In [ ]:
#@title Resposta
cols = [c for c in df1.columns if c.startswith("IN_INTERNET")]
df1[cols].describe()

---

## Filtros 

Tal como séries, DataFrames também podem ser comparados para devolver um DataFrame de booleanos

In [ ]:
df1[["QT_DESKTOP_ALUNO", "IN_INTERNET"]] > 0

De forma análoga, métodos que nós vimos anteriormente também funcionarão em DataFrames

In [ ]:
df1[["QT_DESKTOP_ALUNO", "IN_INTERNET"]].isnull()

In [ ]:
df1[["QT_DESKTOP_ALUNO", "IN_INTERNET"]].notnull()

In [ ]:
df1[["QT_DESKTOP_ALUNO", "IN_INTERNET"]].isin([0, 1])

Entretanto, diferente do que imagina, um dataframe de booleanos não pode ser passado como filtro, uma vez que o filtro de uma DataFrame é realizado no nível linha. Desta forma, temos de reduzir o dataframe a uma série para realizar o filtro

In [ ]:
df1[df1["QT_DESKTOP_ALUNO"] > 0]

In [ ]:
df1[df1["IN_INTERNET"] > 0]

In [ ]:
df1[(df1["QT_DESKTOP_ALUNO"] > 0) | (df1["IN_INTERNET"] > 0)]

Em DataFrames, uma das partes interessantes são operações a nível "eixo". Por exemplo, nós vimos em séries o uso do método "all" para verificar se todos os elementos da série eram verdadeiros:

In [ ]:
(df1["QT_DESKTOP_ALUNO"] > 0).all()

Em DataFrames, por outro lado, o que podemos fazer é mudar a "direção" da operação. A operação acima está realizando a verificação do "all" na vertical, o que significa que o resultado agregado da função é um único valor booleano, porém poderíamos, no caso de um DataFrame, seguir na direção horizontal

In [ ]:
df1[["QT_DESKTOP_ALUNO", "IN_INTERNET"]] > 0

In [ ]:
(df1[["QT_DESKTOP_ALUNO", "IN_INTERNET"]] > 0).all()

In [ ]:
(df1[["QT_DESKTOP_ALUNO", "IN_INTERNET"]] > 0).all(axis="columns")

In [ ]:
(df1[["QT_DESKTOP_ALUNO", "IN_INTERNET"]] > 0).all(axis=1)

In [ ]:
df1[(df1[["QT_DESKTOP_ALUNO", "IN_INTERNET"]] > 0).all(axis=1)]

Quase todos os outros métodos que nós vimos em séries poderão ser manipulados desta forma para operarem sobre colunas em DataFrames

Por fim, uma das sintaxes mais utilizadas para filtros em dataframes é a do .loc

In [ ]:
df1.loc[(df1[["QT_DESKTOP_ALUNO", "IN_INTERNET"]] > 0).all(axis=1)]

A primeira vista não parece nada muito interessante, mas na verdade essa sintaxe nos permite realizar filtros tanto em linhas quanto em colunas

In [ ]:
df1.loc[(df1[["QT_DESKTOP_ALUNO", "IN_INTERNET"]] > 0).all(axis=1), ["ID_ESCOLA", "QT_DESKTOP_ALUNO", "IN_INTERNET"]]

Além disso, um de seus usos mais comuns e a sintaxe do lambda, na qual podemos passar uma função para filtrar o dataframe

In [ ]:
df1.loc[
    lambda f: (f[["QT_DESKTOP_ALUNO", "IN_INTERNET"]] > 0).all(axis=1), 
    ["ID_ESCOLA", "QT_DESKTOP_ALUNO", "IN_INTERNET"]
]

---

**Exercício**

Para as escolas sem acesso a energia elétrica, responda quantas delas tem:

1. Acesso a água potável
2. Acesso a internet
3. Acesso a alimentação na escola
4. Banheiro na escola
5. Biblioteca na escola
6. Acesso a rede de esgoto

In [ ]:
#@title Resposta
cols = [
    "IN_AGUA_POTAVEL",
    "IN_INTERNET",
    "IN_ALIMENTACAO",
    "IN_BANHEIRO",
    "IN_BIBLIOTECA",
    "IN_ESGOTO_INEXISTENTE"
]
df1.loc[lambda f: f["IN_ENERGIA_INEXISTENTE"] == 1, cols].describe().loc["mean"]

---

## Operações 

A maioria as operações que nós realizamos no contexto de bases de dados, na verdade são as operações que nós já vimos entre séries (entre colunas).

In [ ]:
df1["QT_SALAS_UTILIZADAS_FORA"] + df1["QT_SALAS_UTILIZADAS_DENTRO"]

Se nós tentarmos realizar operações entre dataframes precisamos verificar se os mesmos obdecem aos padrões de broadcasting

In [ ]:
# df1[["QT_SALAS_UTILIZADAS_FORA", "QT_SALAS_UTILIZADAS_DENTRO"]] / df1["QT_SALAS_EXISTENTES"]
# Memory Error

Mais comumente, operações com múltiplas colunas serão realizadas utilizando os métodos de operação matemática

In [ ]:
df1[["QT_SALAS_UTILIZADAS_FORA", "QT_SALAS_UTILIZADAS_DENTRO"]].multiply(df1["QT_SALAS_UTILIZADAS"], axis="rows")

---

**Exercício**

Calcule o % de cada equipamento (colunas com "QT_EQUIP") em relação aos equipamentos totais (veja a função np.nansum)

In [ ]:
#@title Resposta
cols = [c for c in df1.columns if c.startswith("QT_EQUIP")]
df1[cols].divide(
    pd.Series(np.nansum(df1[cols].values, axis=1), index=df1.index), 
    axis="rows"
)

---

## Adição de Dados

Adicionar uma nova coluna para um dataframe é algo simples. Basta usar a sintaxe de chaves e dar um nome para a coluna. Mas cuidado, se a coluna já existir ela irá substituir seu conteúdo

In [ ]:
df1["PCT_SALAS_ACESSIVEIS"]

In [ ]:
df1["PCT_SALAS_ACESSIVEIS"] = df1["QT_SALAS_UTILIZADAS_ACESSIVEIS"] / df1["QT_SALAS_UTILIZADAS"]

In [ ]:
df1["PCT_SALAS_ACESSIVEIS"]

Podemos ainda criar colunas com base em arrays, listas ou em valores fixos

In [ ]:
df1["PCT_SALAS_ACESSIVEIS"] = 1

In [ ]:
df1["PCT_SALAS_ACESSIVEIS"]

In [ ]:
df1["PCT_SALAS_ACESSIVEIS"] = int(df1.shape[0] / 3) * [1, 2, 3]

In [ ]:
df1["PCT_SALAS_ACESSIVEIS"]

In [ ]:
df1["PCT_SALAS_ACESSIVEIS"] = (df1["QT_SALAS_UTILIZADAS_ACESSIVEIS"] / df1["QT_SALAS_UTILIZADAS"]).values

In [ ]:
df1["PCT_SALAS_ACESSIVEIS"]

DataFrames também podem ser alterados em determinadas linhas de acordo com filtros, apesar do pandas nos devolver um aviso, uma vez que essa operação pode não ter sido realizada tal qual era esperado

In [ ]:
v = df1.loc[lambda f: f["QT_SALAS_UTILIZADAS"] == 1]
display(v["PCT_SALAS_ACESSIVEIS"])
v["PCT_SALAS_ACESSIVEIS"] = -1
display(v["PCT_SALAS_ACESSIVEIS"])

In [ ]:
df1.loc[lambda f: f["QT_SALAS_UTILIZADAS"] == 1, "PCT_SALAS_ACESSIVEIS"]

In [ ]:
v = df1.iloc[:10]
display(v["PCT_SALAS_ACESSIVEIS"])
v["PCT_SALAS_ACESSIVEIS"] = -1
display(v["PCT_SALAS_ACESSIVEIS"])
display(df1.loc[:10, "PCT_SALAS_ACESSIVEIS"])

O aviso acima é feito, porque o slice do dataframe é uma referência ao dataframe original, de forma que modifica-lo significa modificar os dados originais. Uma maneira mais adequada de fazer a operação acima seria

In [ ]:
df1.loc[lambda f: f["QT_SALAS_UTILIZADAS"] == 1, "PCT_SALAS_ACESSIVEIS"] = -1
df1.loc[lambda f: f["QT_SALAS_UTILIZADAS"] == 1, "PCT_SALAS_ACESSIVEIS"]

No geral, todavia, é recomendado, ao criar uma nova coluna, criar uma cópia do dataframe original, uma vez que muitas vezes queremos manter o mesmo inalterado e também ajuda a facilitar o processo de debuging. Para isso o pandas nos fornece a sintaxe do assign

In [ ]:
df4 = df1.assign(
    PCT_SALAS_ACESSIVEIS=lambda f: np.where(
        f["QT_SALAS_UTILIZADAS"] == 1,
        -1,
        (f["QT_SALAS_UTILIZADAS_ACESSIVEIS"] / f["QT_SALAS_UTILIZADAS"])
    )
)

In [ ]:
id(df4)

In [ ]:
id(df1)

O assign não só permite a sintaxe de chaves como feito acima, mas também pode receber dicionários, caso você queira criar nomes de colunas com base em variáveis

In [ ]:
for i in range(1, 6):
    df4 = df4.assign(
        **{
          f"PCT_SALAS_ACESSIVEIS_{i}": lambda f: np.where(
                f["QT_SALAS_UTILIZADAS"] == 1,
                -1,
                (f["QT_SALAS_UTILIZADAS_ACESSIVEIS"] / f["QT_SALAS_UTILIZADAS"])
            )
        }
    )

In [ ]:
df4.columns

Além das operações de geração de colunas, é possível também combinar dataframes e séries pela operação concat. Um de seus usos mais comuns é de expandir um dataframe horizontalmente. Neste caso será feito o match entre dataframe/série pelo índice para se gerar um novo dataframe com as colunas combinadas

In [ ]:
pd.concat([df1["ID_ESCOLA"], df1["PCT_SALAS_ACESSIVEIS"]], axis="columns")

In [ ]:
pd.concat([df1[["ID_ESCOLA", "CO_MUNICIPIO"]], df1["PCT_SALAS_ACESSIVEIS"]], axis="columns")

In [ ]:
pd.concat([df1[["ID_ESCOLA", "CO_MUNICIPIO"]], df1[["QT_SALAS_UTILIZADAS", "PCT_SALAS_ACESSIVEIS"]]], axis="columns")

In [ ]:
pd.concat([df1["ID_ESCOLA"], df1["CO_MUNICIPIO"], df1["PCT_SALAS_ACESSIVEIS"]], axis="columns")

Também é possível empilhar dados, realizando a operação na difereção de linhas

In [ ]:
pd.concat([df1["ID_ESCOLA"], df1["CO_MUNICIPIO"], df1["PCT_SALAS_ACESSIVEIS"]], axis="rows")

In [ ]:
pd.concat([df1[["ID_ESCOLA", "CO_MUNICIPIO"]], df1[["QT_SALAS_UTILIZADAS", "PCT_SALAS_ACESSIVEIS"]]], axis="rows")

In [ ]:
pd.concat([df1[["ID_ESCOLA", "CO_MUNICIPIO"]], df1["PCT_SALAS_ACESSIVEIS"]], axis="rows")

In [ ]:
pd.concat([df1[["ID_ESCOLA", "CO_MUNICIPIO"]], df1[["ID_ESCOLA", "CO_MUNICIPIO"]]], axis="rows")

Entretanto, quando queremos empilhar dados o mais comum é utilizarmos o método append

In [ ]:
df1[["ID_ESCOLA", "CO_MUNICIPIO"]].append(df1[["ID_ESCOLA", "CO_MUNICIPIO"]])

A última forma de alteração de dados é por meio do método rename, que permite tanto renomear indices quanto colunas a depender dos parâmetros passados. Vale ressaltar que o método devolve uma cópia do data frame original, e que, para aplicar as modificações sobre o data frame devemos passar o parâmetro inplace=True

In [ ]:
df1[["ID_ESCOLA", "CO_MUNICIPIO"]].rename(columns={"CO_MUNICIPIO": "CO"})

In [ ]:
df1[["ID_ESCOLA", "CO_MUNICIPIO"]].rename(index={0: "I0"})

---

**Exercício**

Calcule o % de cada equipamento (colunas com "QT_EQUIP") em relação aos equipamentos totais (veja a função np.nansum) e depois adicione os campos calculados com os nomes alterados ao invés de QT_EQUIP para PCT_EQUIP a base original

In [ ]:
#@title Resposta
cols = [c for c in df1.columns if c.startswith("QT_EQUIP")]
pct = df1[cols].divide(
    pd.Series(np.nansum(df1[cols].values, axis=1), index=df1.index), 
    axis="rows"
)
pct.rename(columns={c: f"PCT_{c.replace('QT_', '')}" for c in pct}, inplace=True)
concat = pd.concat([df1, pct], axis=1)
concat

---

## Remoção de Dados 

Outro processo comum em dataframes é a remoção de dados. Em séries nós já vimos o método "drop" para remover índices, que funciona exatamente igual para dataframes. É mais comum, porém, queremos remover colunas no lugar de índices

In [ ]:
df1.drop(index=[1, 2, 3])

In [ ]:
df4.drop(
    columns=[
        "PCT_SALAS_ACESSIVEIS_1",
        "PCT_SALAS_ACESSIVEIS_2",
        "PCT_SALAS_ACESSIVEIS_3",
        "PCT_SALAS_ACESSIVEIS_4",
        "PCT_SALAS_ACESSIVEIS_5",
    ]
)

In [ ]:
df4.drop(
    columns=[
        "PCT_SALAS_ACESSIVEIS_1",
        "PCT_SALAS_ACESSIVEIS_2",
        "PCT_SALAS_ACESSIVEIS_3",
        "PCT_SALAS_ACESSIVEIS_4",
        "PCT_SALAS_ACESSIVEIS_5",
    ],
    inplace=True
)

In [ ]:
df4.drop("PCT_SALAS_ACESSIVEI", axis=1)

In [ ]:
df4.drop("PCT_SALAS_ACESSIVEI", axis=1, errors="ignore")

Nós também vimos o método drop_duplicates que permite remover linhas de uma série que tenham valores repetidos. O pandas também pode utilizar o mesmo método, todavia por padrão o método irá eliminar linhas em que todas as colunas tenham valores iguais, sendo que para remover duplicatas em colunas específicas é preciso utilizar o parâmetro subset

In [ ]:
df4.shape

In [ ]:
df4.drop_duplicates()

In [ ]:
df4.drop_duplicates(subset=["CO_MUNICIPIO", "CO_ESCOLA_SEDE_VINCULADA"])

In [ ]:
df4.drop_duplicates(subset=["CO_MUNICIPIO"], keep="last") # inplace=True

O pandas ainda vem com outra funcionalidade comum que é a remoção de linhas ou colunas com base na presença de nulos por meio do método dropna

In [ ]:
df1.dropna()

In [ ]:
df1.dropna(axis="columns")

In [ ]:
df1.dropna(axis="columns", how="all")

In [ ]:
df1.dropna(axis="rows", how="any", subset=["CO_ESCOLA_SEDE_VINCULADA"]) # inplace=True

---

**Exercício**

Calcule o % de cada equipamento (colunas com "QT_EQUIP") em relação aos equipamentos totais (veja a função np.nansum) e depois adicione os campos calculados com os nomes alterados ao invés de QT_EQUIP para PCT_EQUIP a base original removendo as colunas que forem totalmente nulas

In [ ]:
#@title Resposta
cols = [c for c in df1.columns if c.startswith("QT_EQUIP")]
pct = df1[cols].divide(
    pd.Series(np.nansum(df1[cols].values, axis=1), index=df1.index), 
    axis="rows"
)
pct.rename(columns={c: f"PCT_{c.replace('QT_', '')}" for c in pct}, inplace=True)
pct.dropna(axis="columns", how="all", inplace=True)
concat = pd.concat([df1, pct], axis=1)
concat

---

## Preenchimento de Nulos 

Nós já vimos preenchimento de nulos em séries, e para dataframes nada muda, a não ser a habilidade de preencher nulos em múltiplas colunas ao mesmo tempo ou a possibilidade de escolher a direção de preenchimento

In [ ]:
df3.reindex(columns=["ID_ESCOLA", "ANO", "IDEB_AI", "IDEB_AF"])

In [ ]:
df3.reindex(columns=["ID_ESCOLA", "ANO", "IDEB_AI", "IDEB_AF"]).fillna(-1)

In [ ]:
df3.reindex(columns=["ID_ESCOLA", "ANO", "IDEB_AI", "IDEB_AF"]).ffill()

In [ ]:
df3.reindex(columns=["ID_ESCOLA", "ANO", "IDEB_AI", "IDEB_AF"]).bfill()

In [ ]:
df3.reindex(columns=["IDEB_AI", "IDEB_AF"]).ffill(axis=1)

---

**Exercício**

Para escolas em funcionamento (TP_SITUACAO_FUNCIONAMENTO = EM ATIVIDADE) que não são particulares (TP_DEPENDENCIA <> PRIVADA), preenche os valores de TP_CONVENIO_PODER_PUBLICO com "PÚBLICA"

In [ ]:
#@title Resposta
filt = (
    lambda f: (f["TP_SITUACAO_FUNCIONAMENTO"] == "EM ATIVIDADE")
    & (f["TP_DEPENDENCIA"] != "PRIVADA")
)
df1.loc[filt, "TP_CONVENIO_PODER_PUBLICO"] = df1.loc[filt, "TP_CONVENIO_PODER_PUBLICO"].fillna("PÚBLICA")

---

## Funções e Métodos 

Nós vimos múltiplas funções que podem ser utilizadas em séries, como funções estatísticas, cálculos entre linhas e cumulativos. Para DataFrames, em geral, nossos cálculos serão feitos coluna a coluna (o que significa que se mantém a mesma sintaxe)

In [ ]:
df1[["QT_COMP_PORTATIL_ALUNO", "QT_DESKTOP_ALUNO"]].cumsum()

In [ ]:
df1[["QT_COMP_PORTATIL_ALUNO", "QT_DESKTOP_ALUNO"]].diff()

In [ ]:
df1[["QT_COMP_PORTATIL_ALUNO", "QT_DESKTOP_ALUNO"]].mean()

Porém em algumas situações vamos querer alterar o eixo de calculo

In [ ]:
df1[["QT_COMP_PORTATIL_ALUNO", "QT_DESKTOP_ALUNO"]].sum(axis=1)

In [ ]:
df1[["QT_COMP_PORTATIL_ALUNO", "QT_DESKTOP_ALUNO"]].sum(axis="columns")

In [ ]:
df1.sort_values(by=["QT_COMP_PORTATIL_ALUNO", "QT_DESKTOP_ALUNO"], ascending=[True, False])[["QT_COMP_PORTATIL_ALUNO", "QT_DESKTOP_ALUNO"]]

In [ ]:
df1[["QT_COMP_PORTATIL_ALUNO", "QT_DESKTOP_ALUNO"]].replace({0: "!"})

---

**Exercício**

Para as escolas públicas e em atividade, preenche o campo de "QT_PROF_ADMINISTRATIVO" com o valor de média por TP_DEPENDENCIA

In [ ]:
#@title Resposta
mfed = df1.loc[lambda f: f["TP_DEPENDENCIA"] == "FEDERAL", "QT_PROF_ADMINISTRATIVOS"].mean()
mest = df1.loc[lambda f: f["TP_DEPENDENCIA"] == "ESTADUAL", "QT_PROF_ADMINISTRATIVOS"].mean()
mmun = df1.loc[lambda f: f["TP_DEPENDENCIA"] == "MUNICIPAL", "QT_PROF_ADMINISTRATIVOS"].mean()
mpriv = df1.loc[lambda f: f["TP_DEPENDENCIA"] == "PRIVADA", "QT_PROF_ADMINISTRATIVOS"].mean()
df1["QT_PROF_ADMINISTRATIVOS"] = np.where(
    df1["TP_SITUACAO_FUNCIONAMENTO"] != "EM ATIVIDADE", 
    df1["QT_PROF_ADMINISTRATIVOS"],
    np.where(
        df1["TP_DEPENDENCIA"] == "FEDERAL",
        df1["QT_PROF_ADMINISTRATIVOS"].fillna(mfed),
        np.where(
            df1["TP_DEPENDENCIA"] == "ESTADUAL",
            df1["QT_PROF_ADMINISTRATIVOS"].fillna(mest),
            np.where(
                df1["TP_DEPENDENCIA"] == "MUNICIPAL",
                df1["QT_PROF_ADMINISTRATIVOS"].fillna(mmun),
                df1["QT_PROF_ADMINISTRATIVOS"].fillna(mpriv)
            )
        )
    )
)

---

## "Vetorização" 

Nós já vimos em séries que nós temos a habilidade de executar funções proprietárias por meio do apply e do map. Para DataFrames, de forma análoga, teremos métodos semelhantes. O primeiro e já conhecido é o próprio apply.

In [ ]:
df1["QT_COMP_PORTATIL_ALUNO"].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

In [ ]:
df1[["QT_COMP_PORTATIL_ALUNO", "QT_DESKTOP_ALUNO"]].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

Ao aplicar o apply sobre um dataframe o pandas irá percorrer os dados em uma determinada direção e irá passar para a função uma série de dados. No caso do código acima, nós passamos de maneira iterativa a série que corresponde a coluna sendo processada

In [ ]:
df1[["QT_COMP_PORTATIL_ALUNO", "QT_DESKTOP_ALUNO"]].apply(lambda x: print(x.name))

In [ ]:
df1[["QT_COMP_PORTATIL_ALUNO", "QT_DESKTOP_ALUNO"]].apply(lambda x: x.mean())

Como vimos antes, poderemos executar o apply percorrendo o dataframe de uma maneira diferente

In [ ]:
(
    df1[["QT_COMP_PORTATIL_ALUNO", "QT_DESKTOP_ALUNO"]]
    .head(3)
)

In [ ]:
(
    df1[["QT_COMP_PORTATIL_ALUNO", "QT_DESKTOP_ALUNO"]]
    .head(3)
    .apply(lambda x: print(x), axis="columns")
)

Ao fazer isso o pandas ainda irá passar uma série para a função, só que desta vez a série corresponderá as cada linha de dados, na qual o nome dos índices serão as colunas do dataframe

In [ ]:
(
    df1.head(3)
    .apply(
        lambda x: x[["QT_COMP_PORTATIL_ALUNO", "QT_DESKTOP_ALUNO"]].max(), 
        axis="columns"
    )
)

Nós também temos o método applymap, uma operação que aplica uma função elemento a elemento de um determinado data frame (equivalente do "map" para séries)

In [ ]:
df1[["QT_COMP_PORTATIL_ALUNO", "QT_DESKTOP_ALUNO"]].applymap(lambda x: len(str(x)), na_action="ignore")

Por fim o último método que temos a nossa disposição para aplicar sobre data frames é o "transform". A primeira vista ele parace ser igual ao apply, mas com algumas diferenças:

1. Permite passar o nome da função

In [ ]:
df1[["QT_COMP_PORTATIL_ALUNO", "QT_DESKTOP_ALUNO"]].transform("sqrt")

In [ ]:
df1[["QT_COMP_PORTATIL_ALUNO", "QT_DESKTOP_ALUNO"]].apply(np.sqrt)

2. Permite passar um dicionário de transformação por coluna (série)

In [ ]:
df1.transform({
    "QT_COMP_PORTATIL_ALUNO": lambda x: (x - x.min()) / (x.max() - x.min()),
    "QT_DESKTOP_ALUNO": "sqrt"
})

3. Não permite gerar valores agregados

In [ ]:
df1[["QT_COMP_PORTATIL_ALUNO", "QT_DESKTOP_ALUNO"]].transform(lambda x: x.mean())

4. Não permite manipular múltiplas direções

In [ ]:
(
    df1.head(3)
    .transform(
        lambda x: x[["QT_COMP_PORTATIL_ALUNO", "QT_DESKTOP_ALUNO"]].max(), 
        axis="columns"
    )
)

5. Não pode ser usado em groupby (veremos mais a frente)

---

**Exercício**

Realize uma transformação por normalização padrão para as colunas do tipo QT, apenas para as colunas que tem algum dado disponível

In [ ]:
#@title Resposta
(
    df1[[c for c in df1.columns if c.startswith("QT_")]]
    .dropna(how="all", axis="columns")
    .apply(lambda x: (x - x.mean()) / x.std())
)

---

## Agrupamento 

Um dos principais calculos que nós temos que realizar em bases de dados é a de agrupar informações. Por exemplo, suponhamos que queiramos saber quantos desktops de aluno cada escola tem em média de acordo com a depêndencia administrativa, como poderíamos fazer isso sem filtrar múltiplas vezes o dataset?

In [ ]:
df1.groupby(["TP_DEPENDENCIA"])["QT_DESKTOP_ALUNO"].mean()

A operação do groupby consiste no equivalente aos seguintes passos:

1. Divida o dataframe em dataframe menores para cada valor da seleção de colunas
> Filtre o data frame para TP_DEPENDENCIA = ESTADUAL
2. Seleciona os campos a serem agregados do data frame
> Selecione o campo QT_DESKTOP_ALUNO
3. Aplique a função de agregação na base resultante
> mean de QT_DESKTOP_ALUNO

Com base nisso, o groupby oferece uma série de variações em termos de chaves, colunas e formas de agregação:

- Múltiplas chaves

In [ ]:
df1.groupby(["TP_SITUACAO_FUNCIONAMENTO", "TP_DEPENDENCIA"])["QT_DESKTOP_ALUNO"].mean()

- Múltiplos campos

In [ ]:
df1.groupby(["TP_SITUACAO_FUNCIONAMENTO", "TP_DEPENDENCIA"])[["QT_COMP_PORTATIL_ALUNO", "QT_DESKTOP_ALUNO"]].mean()

- Transformações especiais

In [ ]:
df1.groupby(["TP_SITUACAO_FUNCIONAMENTO", "TP_DEPENDENCIA"])[["QT_COMP_PORTATIL_ALUNO", "QT_DESKTOP_ALUNO"]].apply(lambda x: x.max() - x.min())

- Aplicar transformações específicas por coluna

In [ ]:
df1.groupby(["TP_SITUACAO_FUNCIONAMENTO", "TP_DEPENDENCIA"]).agg({
    "QT_COMP_PORTATIL_ALUNO": "mean",
    "QT_DESKTOP_ALUNO": lambda x: x.max() - x.min()
})

- Aplicar múltiplas transformações ao mesmo tempo

In [ ]:
df1.groupby(["TP_SITUACAO_FUNCIONAMENTO", "TP_DEPENDENCIA"]).agg({
    "QT_COMP_PORTATIL_ALUNO": [
        "count", "mean", "std", "min", lambda x: x.quantile(0.25), "median", lambda x: x.quantile(0.75), "max"
    ],
    "QT_DESKTOP_ALUNO": [
        "count", "mean", "std", "min", lambda x: x.quantile(0.25), "median", lambda x: x.quantile(0.75), "max"
    ],
})

- Aplicar transformações nomeadas

In [ ]:
df1.groupby(["TP_SITUACAO_FUNCIONAMENTO", "TP_DEPENDENCIA"]).agg(
    QT_COMP_PORTATIL_ALUNO_COUNT=pd.NamedAgg("QT_COMP_PORTATIL_ALUNO", "count"),
    QT_COMP_PORTATIL_ALUNO_MEAN=pd.NamedAgg("QT_COMP_PORTATIL_ALUNO", "mean"),
    QT_COMP_PORTATIL_ALUNO_STD=pd.NamedAgg("QT_COMP_PORTATIL_ALUNO", "std"),
    QT_COMP_PORTATIL_ALUNO_MIN=pd.NamedAgg("QT_COMP_PORTATIL_ALUNO", "min"),
    QT_COMP_PORTATIL_ALUNO_Q1=pd.NamedAgg("QT_COMP_PORTATIL_ALUNO", lambda x: x.quantile(0.25)),
    QT_COMP_PORTATIL_ALUNO_Q2=pd.NamedAgg("QT_COMP_PORTATIL_ALUNO", "median"),
    QT_COMP_PORTATIL_ALUNO_Q3=pd.NamedAgg("QT_COMP_PORTATIL_ALUNO", lambda x: x.quantile(0.75)),
    QT_COMP_PORTATIL_ALUNO_MAX=pd.NamedAgg("QT_COMP_PORTATIL_ALUNO", "max"),
)

Além do groupby outra maneira de agrupar os dados é por meio de pivot tables. O processa de tratamento dos dados será o mesmo, entretanto haverá uma mudança na disposição dos resultados, na qual a chave dos campos pode ser, além de passada em linhas, também em colunas

In [ ]:
df1.pivot_table(
    index="TP_DEPENDENCIA",
    columns="IN_EXAME_SELECAO",
    values="QT_DESKTOP_ALUNO",
    aggfunc="mean"
)

As pivot tables são mais limitadas do que groupby's em funcionalidades, entretanto ainda podemos aumentar a quantidade de informação disponibilizada passando listas para qualquer um dos valores de index, columns, values ou aggfunc

In [ ]:
df1.pivot_table(
    index=["TP_REDE_LOCAL", "TP_DEPENDENCIA"],
    columns=["IN_ALIMENTACAO", "IN_EXAME_SELECAO"],
    values=["QT_DESKTOP_ALUNO", "QT_COMP_PORTATIL_ALUNO"],
    aggfunc={"QT_DESKTOP_ALUNO": "mean", "QT_COMP_PORTATIL_ALUNO": lambda x: x.max() - x.min()}
)

**IMPORTANTE**: Tanto o groupby quanto o pivot table removem da agregação as chaves que possuem valores nulos

In [ ]:
df1["IN_EXAME_SELECAO"].shape

In [ ]:
(
    df1.assign(IN_EXAME_SELECAO=lambda f: f["IN_EXAME_SELECAO"].fillna(-1))
    .groupby(["IN_EXAME_SELECAO"])["ID_ESCOLA"]
    .count()
)

Vocês devem ter notado que nas operações de agregação acima, nós tivemos como resultado novos data frames com índices que equivaliam a combinação de campos nas chaves selecionadas. Essa combinação entre múltiplos campos gerou "hierarquias" nos índices dos data frames, o que equivale a uma estrutura conhecida como "MultiIndex"

In [ ]:
g = df1.groupby(["TP_SITUACAO_FUNCIONAMENTO", "TP_DEPENDENCIA"])[["QT_COMP_PORTATIL_ALUNO", "QT_DESKTOP_ALUNO"]].apply(lambda x: x.max() - x.min())
g.index

Os MultiIndex podem ser mapeados com as mesmas funções de .loc, .iloc, .ix, entretanto ao invés de receber um valor, eles recebem uma tupla de valores.

In [ ]:
g.loc[("EM ATIVIDADE", "ESTADUAL")]

In [ ]:
g.iloc[3]

Além disso ao serem percorridos com um for loop é esperado que tenhamos que desempacotar múltiplos valores de acordo com a combinação de campos

In [ ]:
for f, d in g.index:
    print(f, d)

Na enorme maioria das situações, não será do nosso interesse trabalhar com MultiIndex, por conta disso nós iremos realizar uma de três opções:

1. Vamos "resetar" os índices

In [ ]:
(
    df1.groupby(["TP_SITUACAO_FUNCIONAMENTO", "TP_DEPENDENCIA"])[["QT_COMP_PORTATIL_ALUNO", "QT_DESKTOP_ALUNO"]]
    .apply(lambda x: x.max() - x.min())
    .reset_index()
)

In [ ]:
df1.pivot_table(
    index=["TP_REDE_LOCAL", "TP_DEPENDENCIA"],
    columns=["IN_ALIMENTACAO", "IN_EXAME_SELECAO"],
    values=["QT_DESKTOP_ALUNO", "QT_COMP_PORTATIL_ALUNO"],
    aggfunc={"QT_DESKTOP_ALUNO": "mean", "QT_COMP_PORTATIL_ALUNO": lambda x: x.max() - x.min()}
).reset_index()

In [ ]:
df1.pivot_table(
    index=["TP_REDE_LOCAL", "TP_DEPENDENCIA"],
    columns=["IN_ALIMENTACAO", "IN_EXAME_SELECAO"],
    values=["QT_DESKTOP_ALUNO", "QT_COMP_PORTATIL_ALUNO"],
    aggfunc={"QT_DESKTOP_ALUNO": "mean", "QT_COMP_PORTATIL_ALUNO": lambda x: x.max() - x.min()}
).reset_index(col_level=2).columns

2. Vamos remover um dos níveis

In [ ]:
df1.pivot_table(
    index=["TP_REDE_LOCAL", "TP_DEPENDENCIA"],
    columns=["IN_ALIMENTACAO", "IN_EXAME_SELECAO"],
    values=["QT_DESKTOP_ALUNO", "QT_COMP_PORTATIL_ALUNO"],
    aggfunc={"QT_DESKTOP_ALUNO": "mean", "QT_COMP_PORTATIL_ALUNO": lambda x: x.max() - x.min()}
).reset_index(col_level=2).droplevel(0, axis=1)

3. Vamos renomear os índices

In [ ]:
pv = df1.pivot_table(
    index=["TP_REDE_LOCAL", "TP_DEPENDENCIA"],
    columns=["IN_ALIMENTACAO", "IN_EXAME_SELECAO"],
    values=["QT_DESKTOP_ALUNO", "QT_COMP_PORTATIL_ALUNO"],
    aggfunc={"QT_DESKTOP_ALUNO": "mean", "QT_COMP_PORTATIL_ALUNO": lambda x: x.max() - x.min()}
)
pv.columns = [
    f"{metrica}_ALIM={int(alim)}_EXAM={int(selec)}"
    for metrica, alim, selec in pv.columns
]
pv.reset_index(inplace=True)
pv

---

**Exercício**

Classifique as escolas em quartis de acordo com a quantidade de Desktops de aluno que os mesmos possuem, depois calcule a distribuição de escolas por depêndencia administrativa em cada quartil (veja a função pd.qcut)

In [ ]:
#@title Resposta
pv = (
    df1.assign(QUARTIL=lambda f: pd.qcut(f["QT_SALAS_UTILIZADAS"], 4, labels=["Q1", "Q2", "Q3", "Q4"]))
    .pivot_table(index="TP_DEPENDENCIA", columns="QUARTIL", values="ID_ESCOLA", aggfunc="count")
)
pv.divide(pv.sum(axis=1), axis=0)

---

## Melt 

Outra operação comum no universo de dados do pandas é a de realizar o "melt" (derretimento) dos dados. Isto é, a operação de converter colunas em linhas (quase que o inverso da pivot table). Por exemplo, tomando a base de IDEB, suponhamos que nós queiramos criar uma base com as notas de IDEB independente do tipo, ou seja, queremos uma base com ID_ESCOLA, ANO, TIPO_IDEB, NOTA

In [ ]:
df3.shape

In [ ]:
588485 * 3

In [ ]:
(
    df3.melt(
        id_vars=["ID_ESCOLA", "ANO"],
        value_vars=["IDEB_AI", "IDEB_AF", "IDEB_EM"],
        var_name="TIPO_IDEB",
        value_name="NOTA"
    )
    .assign(TIPO_IDEB=lambda f: f["TIPO_IDEB"].str[5:])
    .dropna()
    .pivot_table(index=["TIPO_IDEB"], columns="ANO", values="NOTA", aggfunc="mean")
)

---

**Exercício**

Tomando a base de escolas, crie uma base de "equipamentos", na qual teremos o ID_ESCOLA, TIPO_EQUIP e QT_EQUIP

In [ ]:
#@title Resposta
(
    df1.melt(
        id_vars=["ID_ESCOLA"],
        value_vars=[c for c in df1 if c.startswith("QT_EQUIP")],
        var_name="TIPO_EQUIP",
        value_name="QT_EQUIP"
    )
    .assign(TIPO_EQUIP=lambda f: f["TIPO_EQUIP"].str[9:])
    .dropna()
)

---

## Join (merge)

"Join" refere-se a operação de junção de duas base de dados. Essa operação é muito popular e conhecida por esse nome no SQL, na qual selecionamos duas tabelas, um conjunto de chaves (e.g. campos comuns entre as duas tabelas) e fazemos a junção das tabelas por essas chave. Essa junção pode ser feita de 4 maneiras:
1. LEFT JOIN: Mantem-se a estrutura da tabela "esquerda" e adiciona os dados comuns da tabela "direita"
2. RIGHT JOIN: Mantem-se a estrutura da tabela "direita" e adiciona os dados comuns da tabela "esquerda"
3. INNER JOIN: Mantem-se apenas os dados onde há intersecção entre as duas tabelas
4. OUTER JOIN: Cria-se uma tabela com os dados das duas tabelas

<img src=https://www.thecrazyprogrammer.com/wp-content/uploads/2019/05/Joins-in-SQL-Inner-Outer-Left-and-Right-Join.jpg width=400 height=300/>

No pandas essa operação é conhecida como "merge", mas segue exatamente as mesmas propriedades. Por exemplo, suponhamos que nós queiramos adicionar as informações de IDEB a base de escola:

In [ ]:
df1.merge(
    df3.loc[lambda f: f["ANO"] == 2019, ["ID_ESCOLA", "IDEB_AI"]],
    left_on=["ID_ESCOLA"],
    right_on=["ID_ESCOLA"],
    how="left"
)

Mescle DataFrame ou objetos Series nomeados com uma junção de estilo de banco de dados.

Um objeto Series nomeado é tratado como um DataFrame com uma única coluna nomeada.

A junção é feita em colunas ou índices. Se juntar colunas em colunas, os índices DataFrame serão ignorados. Caso contrário, se juntando índices em índices ou índices em uma coluna ou colunas, o índice será transmitido. Ao realizar uma mesclagem cruzada, nenhuma especificação de coluna para mesclar é permitido.

**Parâmetros**

right: DataFrame ou objeto de série nomeado para mesclar.

how: {'left', 'right', 'outer', 'inner', 'cross'}, padrão 'inner'
> Tipo de mesclagem a ser executada.
- left: use apenas as teclas do quadro esquerdo, semelhante a uma junção externa esquerda SQL; preservar a ordem das chaves.
- right: use apenas as chaves do quadro direito, semelhante a uma junção externa direita SQL; preservar a ordem das chaves.
- outer: use a união de chaves de ambos os quadros, semelhante a um SQL full outer. Junte; classifique as chaves lexicograficamente.
- inner: usa a interseção de chaves de ambos os quadros, semelhante a um SQL interno. Junte; preservar a ordem das teclas esquerdas.
- cross: cria o produto cartesiano de ambos os frames, preserva o pedido das teclas esquerdas.

on: string ou lista
> Nomes de coluna ou índice para unir. Estes devem ser encontrados em ambos DataFrames. Se `on` for None e não estiver mesclando os índices, então este vai mesclar em todas as colunas comuns nos DataFrames.

left_on: string ou list, ou array-like
> Nomes de coluna ou índice para unir no DataFrame esquerdo. Também pode ser uma matriz ou lista de matrizes do comprimento do DataFrame esquerdo. Essas matrizes são tratadas como se fossem colunas.

right_on: string ou list, ou array-like
> Nomes de coluna ou índice para unir no DataFrame esquerdo. Também pode ser uma matriz ou lista de matrizes do comprimento do DataFrame direito. Essas matrizes são tratadas como se fossem colunas.

left_index: bool, padrão False
> Use o índice da esquerda DataFrame como a(s) chave(s) de junção. Se for um MultiIndex, o número de chaves no outro DataFrame (seja o índice ou um número de colunas) deve corresponder ao número de níveis.

right_index: bool, padrão False
> Use o índice do DataFrame direito como a chave de junção. Mesmas ressalvas que left_index.

sort: bool, padrão False
> Classifique as chaves de junção lexicograficamente no DataFrame de resultado. Se falso, a ordem das chaves de junção depende do tipo de junção (parâmetro how).

suffixes: list-like, padrão é ("\_x", "\_y")
> Uma tupla de comprimento 2 em que cada elemento é opcionalmente uma string indicando o sufixo a ser adicionado aos nomes das colunas sobrepostas em `esquerda` e` direita` respectivamente. Passe um valor de `Nonr` em vez disso de uma string para indicar que o nome da coluna à esquerda ou `right` deve ser deixado como está, sem sufixo. Pelo menos um dos os valores não devem ser None.

copy: bool, padrão True
> Se False, tenta evitar gerar uma cópia

indicator: bool ou str, padrão False
> Se True, adiciona uma coluna ao DataFrame de saída chamada "\_merge" com informações sobre a origem de cada linha. A coluna pode receber um diferente nome fornecendo um argumento de string. A coluna terá um categórico digite com o valor de "left_only" para observações cuja chave de mesclagem apenas aparece no DataFrame esquerdo, "right_only" para observações cuja chave de mesclagem aparece apenas no DataFrame correto, e "ambos" se a chave de mesclagem da observação for encontrada em ambos os DataFrames.

validate: str, opcional
> Se especificado, verifica se a mesclagem é do tipo especificado.
- "one_to_one" ou "1:1": verifique se as chaves de mesclagem são exclusivas em ambos conjuntos de dados esquerdo e direito.
- "one_to_many" ou "1:m": verifique se as chaves de mesclagem são exclusivas à esquerda conjunto de dados.
- "many_to_one" ou "m:1": verifique se as chaves de mesclagem são exclusivas à direita conjunto de dados.
- "many_to_many" ou "m:m": permitido, mas não resulta em verificações.

In [ ]:
df1.merge(
    df3.loc[lambda f: f["ANO"] == 2019, ["ID_ESCOLA", "IDEB_AI"]],
    on=["ID_ESCOLA"],
    how="left",
)

In [ ]:
df1.merge(
    df3.loc[lambda f: f["ANO"] == 2019, ["ID_ESCOLA", "IDEB_AI"]],
    on=["ID_ESCOLA"],
    how="left",
).merge(
    df3.loc[lambda f: f["ANO"] == 2019, ["ID_ESCOLA", "IDEB_AI"]],
    on=["ID_ESCOLA"],
    how="left",
    suffixes=("", "_NOVO")
)

In [ ]:
df1.merge(
    df3.loc[lambda f: f["ANO"] == 2019, ["ID_ESCOLA", "IDEB_AI"]],
    on=["ID_ESCOLA"],
    how="left",
    indicator=True
)["_merge"].value_counts()

In [ ]:
df2.merge(
    df1.reindex(columns=["ID_ESCOLA", "CO_MUNICIPIO"]),
    how="left",
    validate="1:1"
)

In [ ]:
df2.merge(
    df1.reindex(columns=["ID_ESCOLA", "CO_MUNICIPIO"]),
    how="left",
    validate="m:1"
)

Um dos casos mais comuns de erro em merge ocorre quando algum data frame tem dados duplicados, então lembre-se de, quando possível, adicionar um métodod e validate ou de checar se o tamanho final do data frame corresponde a suas expectativas, uma vez que dados duplicados gerarão duplicatas nos dados finais

In [ ]:
df1.reindex(columns=["ID_ESCOLA", "CO_MUNICIPIO"]).merge(
    df2.reindex(columns=["ID_ESCOLA", "ID_TURMA"]),
    how="outer",
    validate="1:m"
)

---

**Exercício**

Calcule o valor da nota média de IDEB dos anos iniciais por depêndencia administrativa ponderado pelo número de turmas em cada escola e o número médio total

In [ ]:
#@title Resposta
(
    df1.loc[lambda f: f["TP_SITUACAO_FUNCIONAMENTO"] == "EM ATIVIDADE"]
    .reindex(columns=["ID_ESCOLA", "TP_DEPENDENCIA"])
    .merge(df3.loc[lambda f: f["ANO"] == 2019, ["ID_ESCOLA", "IDEB_AI"]].dropna())
    .merge(
        df2.groupby(["ID_ESCOLA"])["ID_TURMA"]
        .nunique()
        .reset_index()
    )
    .assign(PROD=lambda f: f["IDEB_AI"] * f["ID_TURMA"])
    .groupby(["TP_DEPENDENCIA"]).agg({
        "IDEB_AI": "mean",
        "ID_TURMA": "sum",
        "PROD": "sum",
    })
    .assign(IDEB_AI_POND=lambda f: f["PROD"] / f["ID_TURMA"])
    .drop(columns=["PROD", "ID_TURMA"])
)

---

## Operações Avançadas 

Nós já vimos em séries as operações de "rolling" e "interpolate". Em DataFrames, nada muda, exceto (como sempre) a nossa habilidade de trabalhar com várias colunas ao mesmo tempo ou de mudar a direção dos cálculos

In [ ]:
df3.reindex(columns=["ID_ESCOLA", "ANO", "IDEB_AI", "IDEB_AF"])

In [ ]:
(
    df3.reindex(columns=["ID_ESCOLA", "ANO", "IDEB_AI", "IDEB_AF"])
    .rolling(2)
    .mean()
)

In [ ]:
(
    df3.reindex(columns=["ID_ESCOLA", "ANO", "IDEB_AI", "IDEB_AF"])
    .rolling(2, axis=1)
    .mean()
)

In [ ]:
(
    df3.reindex(columns=["ID_ESCOLA", "ANO", "IDEB_AI", "IDEB_AF"])
    .interpolate()
)

Uma outra funcionalidade relevante é a de aplicar operações em groupby. Como nós descrevemos o groupby é como se fosse um filtro iterativo sobre a base, na qual é aplicada uma determinada função. Desta forma é possível aplicar qualquer tipo de função sobre o groupby incluindo operações que não necessariamente gerem um resultado agregado.

Por exemplo, veja a operação acima, note que a escola 53082001 ganhou uma nota de IDEB_AI, mesmo não tendo nenhum registro anterior. Isso acontece porque a função do interpolate, por padrão, preenche os valores "para frente" com um forward fill, o que significa que estaremos preenchedo os valores de 53082001 com o último valor preenchido nos dados. A maneira ideal de preencher esses dados no exemplo acima seria por escola, portanto em um groupby

In [ ]:
(
    df3.reindex(columns=["ID_ESCOLA", "ANO", "IDEB_AI", "IDEB_AF"])
    .groupby(["ID_ESCOLA"])["IDEB_AI"].apply(
        lambda x: x.interpolate()
    )
)

Como você pode ver, nós aplicamos o groupby sobre ID_ESCOLA ao invés de ID_ESCOLA + ANO, uma vez que queremos que a função interpolate seja aplicada sobre a série de dados de uma determinada escola, enquanto que o par ID_ESCOLA + ANO nos devolveria um único data point.

Veja que o resultado é uma série de dados com os valores interpolados para o IDEB_AI. Para aplicar isso ao dataframe nós teríamos que fazer:

In [ ]:
(
    df3.reindex(columns=["ID_ESCOLA", "ANO", "IDEB_AI", "IDEB_AF"])
    .sort_values(by=["ID_ESCOLA", "ANO"])
    .reset_index(drop=True)
    .assign(IDEB_AI=lambda f: f.groupby(["ID_ESCOLA"])["IDEB_AI"].apply(lambda x: x.interpolate()))
)

Note que nós ordemos os valores e resetamos os índices antes de aplicar as operações. Nós fazemos isso para garantir que não teremos problemas na hora de preencher os valores no dataframe, uma vez que isso é feito por meio do match de índices

Mas o céu é o limite aqui. E se ao invés de gerar uma nova série de dados nós quisessemos produzir um novo data frame com base nos resultados? Nós poderíamos faze-lo da mesma forma, entretanto aqui o assign não seria efetivo

In [ ]:
df3.loc[lambda f: f["ANO"] == 2021].count()

In [ ]:
df3.reindex(columns=["ID_ESCOLA", "ANO", "IDEB_AI", "IDEB_META_AI"])

In [ ]:
def gera_novo_df(df: pd.DataFrame) -> pd.DataFrame:
    for m in ["AI", "AF", "EM"]:
        df[f"IDEB_{m}"] = df[f"IDEB_{m}"]
    
    if (
        df["IDEB_AI"].count() == 0
        and df["IDEB_AF"].count() == 0
        and df["IDEB_EM"].count() == 0
    ):
        df["TIPO"] = "SEM COLETA DE IDEB"
    elif (
        df["IDEB_AI"].count() > 0
        and df["IDEB_AF"].count() == 0
        and df["IDEB_EM"].count() == 0
    ):
        df["TIPO"] = "ANOS INICIAIS"
    elif (
        df["IDEB_AI"].count() == 0
        and df["IDEB_AF"].count() > 0
        and df["IDEB_EM"].count() == 0
    ):
        df["TIPO"] = "ANOS FINAIS"
    elif (
        df["IDEB_AI"].count() == 0
        and df["IDEB_AF"].count() == 0
        and df["IDEB_EM"].count() > 0
    ):
        df["TIPO"] = "ENSINO MÉDIO"
    else:
        df["TIPO"] = "MÚLTIPLOS"
    
    return df.reindex(columns=["ID_ESCOLA", "ANO", "IDEB_AI", "IDEB_AF", "IDEB_EM", "TIPO"])

df3.head(52).groupby(["ID_ESCOLA"]).apply(gera_novo_df)

---

**Exercício**

Obtenha a projeção do IDEB para os anos faltantes com base na razão do IDEB realizado e da meta. Caso algum valor de meta não esteja preenchido, faça a interpolação do mesmo. Faça o mesmo com a razão de meta, caso não tenhamos um IDEB realizado

In [ ]:
#@title Resposta
def gera_novo_df(df: pd.DataFrame) -> pd.DataFrame:
    for m in ["AI", "AF", "EM"]:
        df[f"IDEB_META_{m}"] = df[f"IDEB_META_{m}"].interpolate(limit_direction="both")
        df[f"RAZAO_{m}"] = df[f"IDEB_{m}"] / df[f"IDEB_META_{m}"]
        df[f"RAZAO_{m}"] = df[f"RAZAO_{m}"].interpolate(limit_direction="both")
        df[f"IDEB_{m}"] = df[f"RAZAO_{m}"] * df[f"IDEB_META_{m}"]
    
    return df.reindex(columns=["ID_ESCOLA", "ANO", "IDEB_AI", "IDEB_AF", "IDEB_EM"])
        

(
    df3.sort_values(by=["ID_ESCOLA", "ANO"])
    .reset_index(drop=True)
    .head(52)
    .groupby(["ID_ESCOLA"]).apply(gera_novo_df)
)

---

## Exportação de Dados 

Uma vez que realizamos todos os tratamentos de dados de interesse na base, a última parte do trabalho é sempre exportar os dados resultantes. 

O pandas faz com que isso seja bem simples, basta utilizar algum método do tipo to_{nome do arquivo}

In [ ]:
exportar = df3.reindex(columns=["ID_ESCOLA", "ANO", "IDEB_AI"]).dropna()
exportar

In [ ]:
exportar.to_parquet("drive/MyDrive/dados.parquet", partition_cols=["ANO"])

**CSV**
```
exportar.to_csv("dados.csv", sep=",", decimal=".", encoding="utf-8")

exportar.to_csv("dados.zip", compression="zip")
```

**Excel**
```
exportar.to_excel("dados.xlsx", sheet_name="IDEB_AI", index=False)

with pd.ExcelWriter("dados.xlsx") as writer:
    exportar.to_excel(writer, sheet_name="IDEB_AI", index=False)
```

**Pickle**
```
exportar.to_pickle("dados.pkl")
```

**Parquet**
```
exportar.to_parquet("dados.parquet", partition_cols=["ANO"])
```

---

**Exercício**

Com base na base gerada no exercício anterior, crie um base com uma aba para cada tipo de IDEB, apenas para as escolas que coletam aquele IDEB, com os valores projetados

In [ ]:
#@title Resposta

# calcula a razão
exportar = (
    df3.sort_values(by=["ID_ESCOLA", "ANO"])
    .reset_index(drop=True)
    .assign(
        IDEB_META_AI=lambda f: f.groupby(["ID_ESCOLA"])["IDEB_META_AI"].apply(
            lambda x: x.interpolate(limit_direction="both")
        ),
        IDEB_META_AF=lambda f: f.groupby(["ID_ESCOLA"])["IDEB_META_AF"].apply(
            lambda x: x.interpolate(limit_direction="both")
        ),
        IDEB_META_EM=lambda f: f.groupby(["ID_ESCOLA"])["IDEB_META_EM"].apply(
            lambda x: x.interpolate(limit_direction="both")
        ),
        RAZAO_AI=lambda f: f["IDEB_AI"] / f["IDEB_META_AI"],
        RAZAO_AF=lambda f: f["IDEB_AF"] / f["IDEB_META_AF"],
        RAZAO_EM=lambda f: f["IDEB_EM"] / f["IDEB_META_EM"],
    )
)

# obtém a projeção de IDEB
exportar = exportar.assign(
    RAZAO_AI=lambda f: f.groupby(["ID_ESCOLA"])["RAZAO_AI"].apply(
        lambda x: x.interpolate(limit_direction="both")
    ),
    RAZAO_AF=lambda f: f.groupby(["ID_ESCOLA"])["RAZAO_AF"].apply(
        lambda x: x.interpolate(limit_direction="both")
    ),
    RAZAO_EM=lambda f: f.groupby(["ID_ESCOLA"])["RAZAO_EM"].apply(
        lambda x: x.interpolate(limit_direction="both")
    ),
    IDEB_AI=lambda f: f["IDEB_META_AI"] * f["RAZAO_AI"],
    IDEB_AF=lambda f: f["IDEB_META_AF"] * f["RAZAO_AF"],
    IDEB_EM=lambda f: f["IDEB_META_EM"] * f["RAZAO_EM"],
)

# faz o melt dos dados
exportar = (
    exportar.melt(
        id_vars=["ID_ESCOLA", "ANO"],
        value_vars=["IDEB_AI", "IDEB_AF", "IDEB_EM"],
        var_name="TIPO",
        value_name="IDEB"
    )
    .dropna()
)

# realiza a geração dos arquivos de saída
with pd.ExcelWriter("dados.xlsx") as writer:
    for m in ["AI", "AF", "EM"]:
        (
            exportar.loc[lambda f: f["TIPO"] == f"IDEB_{m}"]
            .drop(columns="TIPO")
            .to_excel(writer, sheet_name=f"IDEB_{m}", index=False)
        )

---